# Calibrate the rtl-sdr dongle

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(httr)

In [3]:
library(yaml)

Source to functions in Functions.ipynb 

In [4]:
eval(parse(text = system2('jupyter', c('nbconvert', 'Functions.ipynb', '--to=script', '--stdout'), stdout = TRUE)))

In [5]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-29 14:17:56 GMT"

# Config

In [6]:
config_list <- tryCatch({
    read_yaml("configuration.yaml")
}, warning = function(w) {
        print(paste0("No configuration data found in ", getwd()))
        list(satellites=list(METEOR_M2=list(frequency=as.integer(137100000), norad_id=40069, priority=4),
                            METEOR_M2_2=list(frequency=as.integer(137100000), norad_id=44387, priority=5),
                            NOAA_15=list(frequency=as.integer(137620000), norad_id=25338 , priority=3),
                            NOAA_18=list(frequency=as.integer(137912500), norad_id=28654, priority=2),
                            NOAA_19=list(frequency=as.integer(137100000), norad_id=33591, priority=1)),
             n2yo_api_key = "&apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ",
             geonames_username = "pgcudahy",
             minimum_observable_elevation = 20,
             station_latitude = -29.53,
             station_longitude = 30.25,
             station_elevation = NULL,
             minimum_observable_elevation = 20,
             localTimeZone = Sys.timezone(),
             clock_offset = NULL)
}, error = function(e) {
        print(paste0("There was an error in trying to load configuration data from ", getwd()))
        list(satellites=list(METEOR_M2=list(frequency=137100000, norad_id=40069, priority=4),
                            METEOR_M2_2=list(frequency=137100000, norad_id=44387, priority=5),
                            NOAA_18=list(frequency=137912500, norad_id=28654, priority=2),
                            NOAA_19=list(frequency=137100000, norad_id=33591, priority=1)),
             n2yo_api_key =" &apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ",
             geonames_username = "pgcudahy",
             minimum_observable_elevation = 20,
             station_latitude = -29.53,
             station_longitude = 30.25,
             station_elevation = NULL,
             minimum_observable_elevation = 20,
             localTimeZone = Sys.timezone(),
             clock_offset = NULL)
    }
)

In [7]:
config_list$station_elevation <- findStationElevation(config_list$station_elevation, 
                                                      config_list$station_latitude, 
                                                      config_list$station_longitude, 
                                                      config_list$geonames_username)

Sanity check that your coordinates are close to a nearby city

In [8]:
nearby_city <- findNearbyCity(config_list$station_latitude, config_list$station_longitude,
                                config_list$geonames_username)

In [9]:
print(paste0("Station is located near ", nearby_city$geonames[[1]]$toponymName, ", ",
    nearby_city$geonames[[1]]$countryName, " at an elevation of ", config_list$station_elevation,
    " meters above sea level"))

[1] "Station is located near Merrivale, South Africa at an elevation of 1167 meters above sea level"


In [10]:
config_list$gain <- findDongleMaxGain(config_list$gain)

In [11]:
config_list$clock_offset <- findClockOffset(config_list$clock_offset, config_list$gain)

In [12]:
write_yaml(config_list, "configuration.yaml")